<a href="https://colab.research.google.com/github/raultyv/Tareas/blob/main/Calc_evap_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
from google.colab import files
import io

# --- Paso 1: Cargar el archivo CSV de entrada ---
print("Por favor, sube tu archivo CSV (donde la columna A es Fecha y Hora, B es Nivel, C es Lluvia)")
uploaded = files.upload() # Esto abrirá una ventana para que selecciones el archivo

# Obtener el nombre del archivo subido automáticamente
if not uploaded:
    print("No se subió ningún archivo. Por favor, sube tu archivo CSV.")
else:
    file_name = next(iter(uploaded))
    print(f"Archivo detectado y cargado: '{file_name}'")

    try:
        # ASUMIENDO: La columna A es Fecha/Hora, B es Nivel, C es Lluvia.
        # No hay encabezado explícito en la primera fila.
        # Solo necesitamos las primeras 3 columnas (índices 0, 1, 2)
        df = pd.read_csv(io.BytesIO(uploaded[file_name]), header=None, usecols=[0, 1, 2])

    except Exception as e:
        print(f"Error al leer el archivo CSV. Asegúrate de que es un archivo CSV válido con 3 columnas principales. Detalles: {e}")
        exit()

    # --- Paso 2: Preprocesar los datos ---
    df.columns = ['Fecha_Hora', 'Nivel_Tanque', 'Lluvia']

    try:
        # **CAMBIO CLAVE AQUÍ: Usamos infer_datetime_format=True y dayfirst=True para intentar DD/MM/YYYY**
        # Si tu formato principal es MM/DD/YYYY, cambia 'dayfirst=True' a 'dayfirst=False'.
        # La presencia de '13/01/2025' sugiere que 'dayfirst=True' es más probable.
        df['Fecha_Hora'] = pd.to_datetime(df['Fecha_Hora'], infer_datetime_format=True, dayfirst=True, errors='coerce')

        if df['Fecha_Hora'].isnull().any():
            print("\n¡Advertencia! Se encontraron fechas no válidas en la columna 'Fecha_Hora'.")
            print("Estas filas serán omitidas o manejadas como valores nulos.")
            print("Ejemplo de filas con fechas no válidas:")
            print(df[df['Fecha_Hora'].isnull()].head().to_string())
            df.dropna(subset=['Fecha_Hora'], inplace=True)
            print(f"Se eliminaron {df['Fecha_Hora'].isnull().sum()} filas con fechas no válidas.")

    except Exception as e:
        print(f"Error crítico al convertir la columna 'Fecha_Hora'. Detalles: {e}")
        exit()

    if not df.empty:
        df = df.sort_values(by='Fecha_Hora')
    else:
        print("El DataFrame está vacío después de limpiar las fechas. No se pueden realizar cálculos.")
        exit()

    # --- LÍNEAS DE DEPURACIÓN PARA REVISAR DATOS ---
    print("\n--- Primeras 10 filas del DataFrame preprocesado ---")
    print(df.head(10).to_string())

    print("\n--- Últimas 10 filas del DataFrame preprocesado ---")
    print(df.tail(10).to_string())

    print("\n--- Verificación de puntos de inicio de los Días de Evaporación (7 AM) ---")
    df_7am = df[df['Fecha_Hora'].dt.time == pd.to_datetime('07:00').time()]
    print(df_7am.to_string())
    # --- FIN DE LÍNEAS DE DEPURACIÓN ---


    # --- Paso 3: Definir los periodos diarios de 7 AM a 7 AM de forma más explícita y robusta ---
    # Creamos una columna que marca el inicio de cada periodo de evaporación
    # Esta columna contendrá la fecha de las 7 AM del día en que inicia el periodo.
    df['Periodo_Evaporacion_Inicio'] = df['Fecha_Hora'].apply(
        lambda x: (x - pd.Timedelta(hours=7)).floor('D') + pd.Timedelta(hours=7)
    )

    # --- Paso 4: Calcular la evaporación y precipitación diaria ---
    final_daily_data = []

    # Obtenemos todos los días de evaporación únicos y los ordenamos
    unique_evaporation_days = sorted(df['Periodo_Evaporacion_Inicio'].unique())

    for day_start_time in unique_evaporation_days:
        # Definir el rango de tiempo para este periodo (ej. 2024-01-01 07:00:00 a 2024-01-02 06:59:59)
        start_of_period = day_start_time
        end_of_period = day_start_time + pd.Timedelta(days=1)

        # Filtrar datos para este periodo
        period_data = df[(df['Fecha_Hora'] >= start_of_period) & (df['Fecha_Hora'] < end_of_period)]

        if period_data.empty:
            continue # Si no hay datos para este periodo, saltar

        # --- Nivel_Inicio: Encontrar la lectura a las 7 AM del día de inicio ---
        nivel_inicio_row = period_data[period_data['Fecha_Hora'] == start_of_period]
        if not nivel_inicio_row.empty:
            nivel_inicio_real = nivel_inicio_row['Nivel_Tanque'].iloc[0]
        else:
            # Si no hay lectura exacta a las 7 AM, tomar la primera lectura disponible en el periodo
            nivel_inicio_real = period_data['Nivel_Tanque'].iloc[0] if not period_data.empty else float('nan')
            print(f"Advertencia: No se encontró lectura exacta de 7 AM para Nivel_Inicio en {day_start_time}. Usando la primera lectura disponible: {nivel_inicio_real}")


        # --- Nivel_Fin: Encontrar la lectura a las 7 AM del día siguiente ---
        # Buscamos la lectura de las 7 AM del día siguiente en todo el DataFrame para ser más robustos
        end_level_row = df[df['Fecha_Hora'] == end_of_period]
        if not end_level_row.empty:
            nivel_fin_real = end_level_row['Nivel_Tanque'].iloc[0]
        else:
            # Si no hay lectura exacta de 7 AM del día siguiente, tomamos la última lectura del periodo actual
            nivel_fin_real = period_data['Nivel_Tanque'].iloc[-1] if not period_data.empty else float('nan')
            print(f"Advertencia: No se encontró lectura exacta de 7 AM para Nivel_Fin en {end_of_period}. Usando la última lectura del periodo anterior: {nivel_fin_real}")


        # Lluvia acumulada en el periodo (desde 7 AM del día X hasta justo antes de 7 AM del día X+1)
        lluvia_periodo = period_data['Lluvia'].sum()

        # Calcular la diferencia y evaporación
        if not pd.isna(nivel_inicio_real) and not pd.isna(nivel_fin_real):
            diferencia_nivel = nivel_fin_real - nivel_inicio_real
            evaporacion_diaria = -diferencia_nivel + lluvia_periodo
        else:
            diferencia_nivel = float('nan')
            evaporacion_diaria = float('nan')

        final_daily_data.append({
            'Dia_Evaporacion': day_start_time,
            'Nivel_Inicio': nivel_inicio_real,
            'Nivel_Fin': nivel_fin_real,
            'Diferencia_Nivel': diferencia_nivel,
            'Lluvia_Acumulada': lluvia_periodo,
            'Evaporacion_Diaria': evaporacion_diaria
        })

    daily_data = pd.DataFrame(final_daily_data)
    # Opcional: Si quieres eliminar los días donde no se pudo calcular inicio o fin, descomenta la línea de abajo.
    # daily_data.dropna(subset=['Nivel_Inicio', 'Nivel_Fin'], inplace=True)


    # --- Paso 5: Mostrar los resultados y guardar en un nuevo CSV para descargar ---
    print("\nEvaporación y Precipitación Diaria (7 AM a 7 AM)")
    print(daily_data[['Dia_Evaporacion', 'Nivel_Inicio', 'Nivel_Fin', 'Diferencia_Nivel', 'Lluvia_Acumulada', 'Evaporacion_Diaria']].to_string())

    output_file_name = 'evaporacion_diaria_resultados.csv'
    daily_data[['Dia_Evaporacion', 'Nivel_Inicio', 'Nivel_Fin', 'Diferencia_Nivel', 'Lluvia_Acumulada', 'Evaporacion_Diaria']].to_csv(output_file_name, index=False)

    files.download(output_file_name)
    print(f"\nEl archivo '{output_file_name}' ha sido generado y está listo para descargar.")

Por favor, sube tu archivo CSV (donde la columna A es Fecha y Hora, B es Nivel, C es Lluvia)


Saving evap_2023.csv to evap_2023.csv
Archivo detectado y cargado: 'evap_2023.csv'

--- Primeras 10 filas del DataFrame preprocesado ---
           Fecha_Hora  Nivel_Tanque  Lluvia
0 2023-01-01 07:00:00         213.0     0.0
1 2023-01-01 07:15:00         213.0     0.0
2 2023-01-01 07:30:00         213.0     0.0
3 2023-01-01 07:45:00         213.0     0.0
4 2023-01-01 08:00:00         213.0     0.0
5 2023-01-01 08:15:00         213.0     0.0
6 2023-01-01 08:30:00         213.0     0.0
7 2023-01-01 08:45:00         213.0     0.0
8 2023-01-01 09:00:00         213.0     0.0
9 2023-01-01 09:15:00         213.0     0.0

--- Últimas 10 filas del DataFrame preprocesado ---
               Fecha_Hora  Nivel_Tanque  Lluvia
34893 2023-12-31 09:30:00         136.9     0.0
34894 2023-12-31 09:45:00         136.9     0.0
34895 2023-12-31 10:00:00         136.9     0.0
34896 2023-12-31 10:15:00         136.9     0.0
34897 2023-12-31 10:30:00         136.9     0.0
34898 2023-12-31 10:45:00         136.

<ipython-input-11-a470622f13dc>:33: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Fecha_Hora'] = pd.to_datetime(df['Fecha_Hora'], infer_datetime_format=True, dayfirst=True, errors='coerce')


Advertencia: No se encontró lectura exacta de 7 AM para Nivel_Fin en 2024-01-01 07:00:00. Usando la última lectura del periodo anterior: 136.8

Evaporación y Precipitación Diaria (7 AM a 7 AM)
        Dia_Evaporacion  Nivel_Inicio  Nivel_Fin  Diferencia_Nivel  Lluvia_Acumulada  Evaporacion_Diaria
0   2023-01-01 07:00:00        213.00     215.20              2.20             9.906               7.706
1   2023-01-02 07:00:00        215.20     212.40             -2.80             0.000               2.800
2   2023-01-03 07:00:00        212.40     211.10             -1.30             1.524               2.824
3   2023-01-04 07:00:00        211.10     212.30              1.20             4.572               3.372
4   2023-01-05 07:00:00        212.30     216.70              4.40             8.636               4.236
5   2023-01-06 07:00:00        216.70     166.90            -49.80             7.112              56.912
6   2023-01-07 07:00:00        166.90     188.50             21.60      

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


El archivo 'evaporacion_diaria_resultados.csv' ha sido generado y está listo para descargar.
